In [1]:
import torch 
import torchmetrics
import torch.nn as nn
from pytorch_lightning import LightningModule,Trainer
from torch.utils.data import TensorDataset,DataLoader

In [7]:
from dataset_ import EEG_inception
import importlib
import models_
importlib.reload(models_)
# Import models from the models module
from models_ import ResNet1D, Bio, Conv1D_v2, EEGInceptionModel, ChronoNet


### instantiating the lighting module for the model 

In [ ]:
from pytorch_lightning import LightningModule
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torchmetrics
from torch.optim.lr_scheduler import StepLR

class ChronoModel(LightningModule):
    def __init__(self, attribute):
        super(ChronoModel, self).__init__()
        self.attribute = attribute
        self.model = attribute["model"] # initialize the model
        self.lr = attribute["lr"]
        self.bs = 64
        self.worker = 1
        self.acc = torchmetrics.Accuracy(task="binary")
     
        self.criterion = nn.BCEWithLogitsLoss()
    
    def forward(self, x):
        return self.model(x)
    
    def configure_optimizers(self):
        # Create Adam optimizer
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        
        # Define a StepLR learning rate scheduler
        scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
        
        # Return both the optimizer and scheduler
        return optimizer, scheduler

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001, weight_decay= )
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
        return {"optimizer": optimizer, "lr_scheduler": scheduler}

    def train_dataloader(self):
        dataset = self.attribute["train_dataset"]
        return DataLoader(dataset, batch_size=self.bs, num_workers=self.worker, shuffle=True)

    def val_dataloader(self):
        dataset = self.attribute["val_dataset"]
        return DataLoader(dataset, batch_size=self.bs, num_workers=self.worker, shuffle=False)

    def training_step(self, batch, batch_idx):
        signal, label = batch
        out = self(signal.float())
        loss = self.criterion(out.flatten(), label.float().flatten())
        preds = (torch.sigmoid(out.flatten()) > 0.5).long()
        # print(preds.)
        acc = self.acc(preds, label.long().flatten())
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        signal, label = batch
        out = self(signal.float())
        loss = self.criterion(out.flatten(), label.float().flatten())
        preds = (torch.sigmoid(out.flatten()) > 0.5).long()
        acc = self.acc(preds, label.long().flatten())
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', acc, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def on_train_epoch_end(self):
        # Retrieve the average training loss
        train_loss = self.trainer.callback_metrics.get("train_loss_epoch", None)
        # Log the loss for each training step
        if train_loss is not None:
            self.print(f"Epoch {self.current_epoch} - Training Loss: {train_loss:.4f}")
    

    def on_validation_epoch_end(self):
        # Retrieve the average validation loss
        val_loss = self.trainer.callback_metrics.get("val_loss", None)
        if val_loss is not None:
            self.print(f"Epoch {self.current_epoch} - Validation Loss: {val_loss:.4f}")


In [4]:
# train_datas
val_dataset =  EEG_inception(kind = "val", normalize= 1)
train_dataset_smote = EEG_inception(kind="train", normalize= 1, balancing="smote")
train_dataset = EEG_inception(kind="train", normalize= 1, balancing="inception")

lrs = [0.0001, 0.00005, 0.001]

should be here 


100%|██████████| 856/856 [00:00<00:00, 1857.53it/s]


(856, 8, 750) shap[e]


100%|██████████| 7650/7650 [00:18<00:00, 422.06it/s] 


(856, 8, 750) in here dataset
val main_job done 1350 856 856


...

(10312, 8, 750) in here dataset
train main_job done 10790 10312 10312


100%|██████████| 19384/19384 [01:46<00:00, 181.85it/s]


(19384, 8, 750) shap[e]


100%|██████████| 7650/7650 [00:09<00:00, 791.40it/s] 


(19384, 8, 750) in here dataset
train main_job done 10790 19384 19384


In [8]:
# Generate a random input tensor of shape (batch_size=3, channels=8, time_steps=900)
x = torch.randn(3, 8, 750)

# Initialize an empty list to store the models
models = []

# 1. Sanity check for ResNet1D
model_1 = ResNet1D()
models.append(model_1)
output_1 = model_1(x)
print(f"Output of ResNet1D model: {output_1.shape}")

# 2. Sanity check for Bio model
model_3 = Bio(input_size=8)
models.append(model_3)
output_3 = model_3(x)
print(f"Output of Bio model: {output_3.shape}")

# 3. Sanity check for Conv1D_v2
model_4 = Conv1D_v2(channels=8)
models.append(model_4)
output_4 = model_4(x)
print(f"Output of Conv1D_v2 model: {output_4.shape}")

# 4. Sanity check for SimplifiedEEGInceptionModel
model_5 = EEGInceptionModel(in_channels=8)
models.append(model_5)
output_5 = model_5(x)
print(f"Output of SimplifiedEEGInceptionModel: {output_5.shape}")

# 5. Sanity check for ChronoNet
model_6 = ChronoNet(8)
models.append(model_6)
output_6 = model_6(x)
print(f"Output of ChronoNet model: {output_6.shape}")


Output of ResNet1D model: torch.Size([3, 1])
Output of Bio model: torch.Size([3, 1])
Output of Conv1D_v2 model: torch.Size([3, 1])
Output of SimplifiedEEGInceptionModel: torch.Size([3, 1])
Output of ChronoNet model: torch.Size([3, 1])


In [9]:
attributes = {
    1: {"model": ResNet1D(), "train_dataset": train_dataset, "val_dataset": val_dataset, "lr": 0.0001},
    2: {"model": ResNet1D(), "train_dataset": train_dataset_smote, "val_dataset": val_dataset, "lr": 0.0001},
    3: {"model": ResNet1D(), "train_dataset": train_dataset, "val_dataset": val_dataset, "lr": 0.0005},
    4: {"model": ResNet1D(), "train_dataset": train_dataset_smote, "val_dataset": val_dataset, "lr": 0.0005},
    
    5: {"model": Bio(input_size=8), "train_dataset": train_dataset, "val_dataset": val_dataset, "lr": 0.0001},
    6: {"model": Bio(input_size=8), "train_dataset": train_dataset_smote, "val_dataset": val_dataset, "lr": 0.0001},
    7: {"model": Bio(input_size=8), "train_dataset": train_dataset, "val_dataset": val_dataset, "lr": 0.0005},
    8: {"model": Bio(input_size=8), "train_dataset": train_dataset_smote, "val_dataset": val_dataset, "lr": 0.0005},
    
    9: {"model": Conv1D_v2(channels=8), "train_dataset": train_dataset, "val_dataset": val_dataset, "lr": 0.0001},
    10: {"model": Conv1D_v2(channels=8), "train_dataset": train_dataset_smote, "val_dataset": val_dataset, "lr": 0.0001},
    11: {"model": Conv1D_v2(channels=8), "train_dataset": train_dataset, "val_dataset": val_dataset, "lr": 0.0005},
    12: {"model": Conv1D_v2(channels=8), "train_dataset": train_dataset_smote, "val_dataset": val_dataset, "lr": 0.0005},
    
    13: {"model": EEGInceptionModel(in_channels=8), "train_dataset": train_dataset, "val_dataset": val_dataset, "lr": 0.0001},
    14: {"model": EEGInceptionModel(in_channels=8), "train_dataset": train_dataset_smote, "val_dataset": val_dataset, "lr": 0.0001},
    15: {"model": EEGInceptionModel(in_channels=8), "train_dataset": train_dataset, "val_dataset": val_dataset, "lr": 0.0005},
    16: {"model": EEGInceptionModel(in_channels=8), "train_dataset": train_dataset_smote, "val_dataset": val_dataset, "lr": 0.0005},
    
    17: {"model": ChronoNet(channel=8), "train_dataset": train_dataset, "val_dataset": val_dataset, "lr": 0.0001},
    18: {"model": ChronoNet(channel=8), "train_dataset": train_dataset_smote, "val_dataset": val_dataset, "lr": 0.0001},
    19: {"model": ChronoNet(channel=8), "train_dataset": train_dataset, "val_dataset": val_dataset, "lr": 0.0005},
    20: {"model": ChronoNet(channel=8), "train_dataset": train_dataset_smote, "val_dataset": val_dataset, "lr": 0.0005}
}

In [ ]:
import os
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning import Trainer




for model_key, attribute in attributes.items():
    print(f"{attribute['train_dataset']}_{attribute['lr']}")

    model_name = attribute["model"].__class__.__name__
    print(model_name, "here")
    # Extract the learning rate (if present)
    lr = attribute.get("lr", None)  # Get learning rate, default to None if not present
    
    # Extract the train and validation datasets
    train_dataset = attribute["train_dataset"]
    val_dataset = attribute["val_dataset"]
    
    # Determine the dataset type (train_dataset_smote or train_dataset)
    dataset_type = 'train_dataset_smote' if train_dataset == train_dataset_smote else 'train_dataset'


    # Define EarlyStopping callback
    early_stopping_callback = EarlyStopping(
        monitor='val_loss',
        patience=100,
        verbose=True,
        mode='min',
        check_finite=True
    )

    print(model_name, "here")
    # Modify ModelCheckpoint callback to use a custom filename with placeholders
    
    
    # Create a model name based on the above information
    model_identifier = f"{model_name}_lr_{lr}_dataset_{dataset_type}"
    
    model = attribute["model"]
    lr = attribute["lr"]
    train_dataset = attribute["train_dataset"]
    dataset_type = 'train_dataset_smote' if train_dataset == train_dataset_smote else 'train_dataset_inception'

    # Print the model identifier for clarity
    print(f"Training model: {model_identifier}")

    checkpoint_callback = ModelCheckpoint(
        dirpath=f"checkpoints_v1/{model_identifier}",
        filename='{epoch}_valacc_{val_acc:.4f}_trainacc_{train_acc:.4f}',
        save_top_k=3,
        verbose=True,
        monitor='val_loss',
        mode='min'
    )

    trainer = Trainer(
        max_epochs=200,
        callbacks=[early_stopping_callback, checkpoint_callback],
    )
    
    # Train the model
    model = ChronoModel(attribute)
    trainer.fit(model)
    print(trainer.callback_metrics)



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


<dataset_.EEG_inception object at 0x0000015D8E6304D0>_0.0001
ResNet1D here
ResNet1D here
Training model: ResNet1D_lr_0.0001_dataset_train_dataset


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params | Mode 
--------------------------------------------------------
0 | model     | ResNet1D          | 1.5 M  | train
1 | acc       | BinaryAccuracy    | 0      | train
2 | criterion | BCEWithLogitsLoss | 0      | train
--------------------------------------------------------
1.5 M     Trainable params
0         Non-trainable params
1.5 M     Total params
5.906     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\zokov\.conda\envs\py3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
c:\Users\zokov\.conda\envs\py3\Lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
c:\Users\zokov\.conda\envs\py3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


...

Epoch 30, global step 9393: 'val_loss' was not in top 3


Epoch 30 - Validation Loss: 0.7267
Epoch 30 - Training Loss: 0.3430


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 31, global step 9696: 'val_loss' was not in top 3


Epoch 31 - Validation Loss: 0.7490
Epoch 31 - Training Loss: 0.2994


In [40]:
print(trainer.callback_metrics['train_loss'])

tensor(0.6296, device='cuda:0')


In [38]:
model.val_loss

AttributeError: 'ChronoModel' object has no attribute 'val_loss'

In [37]:
# After training
print(trainer.logger.version)  # Unique version/run identifier
print(trainer.logger.name)     # Name of the logger
print(trainer.logger.log_dir)  # Full path to logging directory

93
lightning_logs
d:\Vikas\lie_detection\BrainWaves\lightning_logs\version_93
